# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---


🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.


In [13]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyinputplus: filename=pyinputplus-0.2.12-py3-none-any.whl size=11348 sha256=5f5a1430f0f40963bb5c203a3686287a3e26f473e3babe362642ff81c42a5b64
  Stored in directory: /root/.cache/pip/wheels/a3/6b/89/3bec14288af8e0729d088fb41a785e583d49e772f2529c7676
  Created wheel for pysimplevalidate: filename=pysimplevalidate-0.2.12-py3-none-any.whl size=16226 sha256=8a2f7b168c53a2cce6a9a6a36ccdb1c6289399d2325e79a5763b9dfa4c68ac63
  Stored in directory: /root/.cache/pip/wheels/fc/40/7a/4d3b2dc2e80b4b3bbda89eec94fcefd6c8bd1101cc7bb89554
  Created wheel for stdiomask: filename=stdiomask

## 📦 Setup and Configuration
Import required packages and setup environment.

In [16]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'      # this is not working  'http://ollama.serveur.au'  getting error because of this. and can't find any solution.
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: wttr.in


In [17]:
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here

## 🌤️ Weather Data Functions

In [18]:
# Define get_weather_data() function here
def get_weather_data(location, days=2):
    """
    Fetch weather forecast data for a given location and number of days.
    """
    try:
        data = get_weather(location=location, days=days)
        return data
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return None



## 📊 Visualisation Functions

In [19]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
import matplotlib.pyplot as plt

def create_temperature_visualisation(weather_data):
    """
    Plot the daily high and low temperatures.
    """
    days = [d['date'] for d in weather_data['forecast']]
    highs = [d['temperature_high'] for d in weather_data['forecast']]
    lows = [d['temperature_low'] for d in weather_data['forecast']]

    plt.figure(figsize=(8, 5))
    plt.plot(days, highs, label='High Temp', marker='o')
    plt.plot(days, lows, label='Low Temp', marker='o')
    plt.title('Temperature Forecast')
    plt.xlabel('Date')
    plt.ylabel('Temperature (°C)')
    plt.legend()
    plt.grid(True)
    plt.show()

def create_precipitation_visualisation(weather_data):
    """
    Plot the precipitation chance for each day.
    """
    days = [d['date'] for d in weather_data['forecast']]
    precipitation = [d['precipitation_chance'] for d in weather_data['forecast']]

    plt.figure(figsize=(8, 5))
    plt.bar(days, precipitation, color='skyblue')
    plt.title('Precipitation Chance')
    plt.xlabel('Date')
    plt.ylabel('Chance (%)')
    plt.grid(True)
    plt.show()


## 🤖 Natural Language Processing

In [20]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Use AI to extract intent, location, and timeframe from the user's question.
    """
    prompt = f"""
    Extract the following from this weather question:
    - location
    - timeframe (e.g., today, tomorrow)
    - intent (e.g., rain, umbrella, cold, clothing)

    Return the output as a JSON object with keys: location, timeframe, intent.

    Question: "{question}"
    """
    response = get_response(prompt)
    try:
        return eval(response)
    except:
        print("Failed to parse response:", response)
        return None


In [21]:
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a response based on the parsed question and weather data.
    """
    if not parsed_question or not weather_data:
        return "Sorry, I couldn't understand the question or fetch the weather."

    intent = parsed_question['intent']
    timeframe = parsed_question['timeframe']

    forecast = None
    for day in weather_data['forecast']:
        if timeframe.lower() in day['date'].lower() or timeframe.lower() == "tomorrow":
            forecast = day
            break
    if not forecast:
        return "Sorry, I couldn't find the forecast for the requested timeframe."

    if intent == "rain":
        return f"The chance of rain {timeframe} is {forecast['precipitation_chance']}%."
    elif intent == "umbrella":
        chance = forecast['precipitation_chance']
        return "Yes, bring an umbrella." if chance > 50 else "No umbrella needed."
    elif intent == "cold":
        return f"The temperature {timeframe} will be between {forecast['temperature_low']}°C and {forecast['temperature_high']}°C."
    elif intent == "clothing":
        temp = forecast['temperature_low']
        if temp < 10:
            return "Wear warm clothes, it's going to be cold."
        elif temp < 20:
            return "You might want a light jacket."
        else:
            return "Light clothing should be fine."
    else:
        return "I'm not sure how to answer that question."


## 🧭 User Interface

In [23]:
# Define menu functions using pyinputplus or ipywidgets here
import pyinputplus as pyip

def enter_location():
    location = pyip.inputStr(prompt="Enter your location (city name): ")
    print(f"Location set to: {location}")
    return location

def ask_weather_question(location):
    question = pyip.inputStr(prompt="Ask your weather question: ")
    parsed = parse_weather_question(question)
    if parsed:
        weather_data = get_weather_data(location)
        response = generate_weather_response(parsed, weather_data)
        print("🤖 AI says:", response)
    else:
        print("Couldn't understand the question.")


## 🧪 Testing and Examples

In [26]:
# Example 1
location = "Melbourne"
question = "Will it rain tomorrow in Melbourne?"
parsed = parse_weather_question(question)
weather_data = get_weather_data(location)
response = generate_weather_response(parsed, weather_data)
print(f"Question: {question}\nParsed: {parsed}\nResponse: {response}")

# Example 2
question = "Should I wear a jacket tomorrow?"
parsed = parse_weather_question(question)
response = generate_weather_response(parsed, weather_data)
print(f"Question: {question}\nParsed: {parsed}\nResponse: {response}")



Please select one of the following:
1. Enter Location
2. Ask Weather Question
3. Exit
1
Enter your location (city name): melbourne
Location set to: melbourne
Please select one of the following:
1. Enter Location
2. Ask Weather Question
3. Exit
2
Ask your weather question: will it rain tommorow in melbourne


[WARNING] Error during request (attempt 1): 401 Client Error: Unauthorized for url: http://ollama.serveur.au/api/generate


Oops! I got a little tangled up... Let's try that again 😊


[WARNING] Error during request (attempt 2): 401 Client Error: Unauthorized for url: http://ollama.serveur.au/api/generate


Failed to parse response: ❌ Error: 401 Client Error: Unauthorized for url: http://ollama.serveur.au/api/generate
Couldn't understand the question.
Please select one of the following:
1. Enter Location
2. Ask Weather Question
3. Exit
3
Goodbye!


## 🧩 Main Application Logic

In [25]:
# Tie everything together here
def main_menu():
    location = None
    while True:
        choice = pyip.inputMenu(['Enter Location', 'Ask Weather Question', 'Exit'], numbered=True)

        if choice == 'Enter Location':
            location = enter_location()

        elif choice == 'Ask Weather Question':
            if not location:
                print("⚠️ Please enter your location first!")
            else:
                ask_weather_question(location)

        else:
            print("Goodbye!")
            break

# Run the main app
main_menu()


Please select one of the following:
1. Enter Location
2. Ask Weather Question
3. Exit
sydney
'sydney' is not a valid choice.
Please select one of the following:
1. Enter Location
2. Ask Weather Question
3. Exit
Sydney
'Sydney' is not a valid choice.
Please select one of the following:
1. Enter Location
2. Ask Weather Question
3. Exit
1
Enter your location (city name): Sydney
Location set to: Sydney
Please select one of the following:
1. Enter Location
2. Ask Weather Question
3. Exit
2
Ask your weather question: What’s the temperature in Sydney today?


[WARNING] Error during request (attempt 1): 401 Client Error: Unauthorized for url: http://ollama.serveur.au/api/generate


Oops! I got a little tangled up... Let's try that again 😊


[WARNING] Error during request (attempt 2): 401 Client Error: Unauthorized for url: http://ollama.serveur.au/api/generate


Failed to parse response: ❌ Error: 401 Client Error: Unauthorized for url: http://ollama.serveur.au/api/generate
Couldn't understand the question.
Please select one of the following:
1. Enter Location
2. Ask Weather Question
3. Exit
3
Goodbye!
